In [2]:
# Ships classifier using CNN from scratch, with Tensorflow and sklearn

import tensorflow as tf
import numpy as np
import pandas as pd


In [3]:
# Read the Shipsnet json dataset in pandas framework for further processing
df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/shipsnet.json")

In [4]:
# Normalize and reshape the image data
# The pixel values are stored in a column in the data frame titled “data.”
# As is, these pixel values aren’t ready to be processed by a CNN.
# Instead, the new data is converted to a NumPy array and divided by 255 to normalize the values. 
# All 19,200 values should now be some value between 0 and 1. 
# Next the data is reshaped to 80 x 80 x 3 matrix so that it’s formatted as a picture.
df["normalized_data"] = df["data"].apply(lambda x: (np.array(x) / 255).reshape(80, 80, 3))

In [5]:
# Define X and Y
X = df["normalized_data"]
Y = df["labels"]

In [6]:
#Spliting the data into training data and Testing data using Sklearn
# Split the data into training and testing sets. Use a 75/25 split
from sklearn.model_selection import train_test_split
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.25, random_state=42)


In [7]:
# As Pandas Series aren’t accepted in TensorFlow, so the training and testing data are converted into arrays.
# Transform the training and testing data into arrays
X_train = np.array([x for x in X_train])
X_test = np.array([x for x in X_test])
Y_train = np.array([y for y in Y_train])
Y_test = np.array([y for y in Y_test])

In [8]:
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Activation


In [9]:
# To the start the model with sequential ANN
model = models.Sequential()

In [10]:
# Adds the first convolusion layer and followed by max pooling

model.add(layers.Conv2D(32, kernel_size=(3,3), activation= 'relu', input_shape = (80, 80, 3)))
# Max pooling layer

model.add(layers.MaxPool2D((2,2)))

In [11]:
#Additional Hidden Layers
model.add(layers.Conv2D(64,kernel_size=(3,3), activation= 'relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(64,kernel_size=(3,3), activation= 'relu'))

In [12]:
# Flattens the input into a 1D tensor
model.add(layers.Flatten())
# Makes the input more readable for classification
model.add(layers.Dense(64, activation='relu'))
# Classifies - ensure the input in the number of classes, indexed
# at 0
model.add(layers.Dense(1))
# Final activation function
model.add(Activation('sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 78, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [15]:
# Compile the model
# Use binary_crossentropy because there are only 2 classes present

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# The above line simply compiles the model. 
# If there were issues with input/output dimensionality while adding layers, 
# the program will let you know at this step.

In [16]:
# Training the model 
gen_model = model.fit(x = X_train, y= Y_train, epochs =20 , validation_data= (X_test, Y_test))

Epoch 1/20
94/94 [==============================] - 34s 16ms/step - loss: 0.4876 - accuracy: 0.8172 - val_loss: 0.1990 - val_accuracy: 0.9250
Epoch 2/20
94/94 [==============================] - 1s 11ms/step - loss: 0.1667 - accuracy: 0.9351 - val_loss: 0.0912 - val_accuracy: 0.9680
Epoch 3/20
94/94 [==============================] - 1s 11ms/step - loss: 0.0768 - accuracy: 0.9696 - val_loss: 0.0575 - val_accuracy: 0.9830
Epoch 4/20
94/94 [==============================] - 1s 11ms/step - loss: 0.0549 - accuracy: 0.9816 - val_loss: 0.0478 - val_accuracy: 0.9870
Epoch 5/20
94/94 [==============================] - 1s 11ms/step - loss: 0.0412 - accuracy: 0.9873 - val_loss: 0.0626 - val_accuracy: 0.9800
Epoch 6/20
94/94 [==============================] - 1s 11ms/step - loss: 0.0268 - accuracy: 0.9902 - val_loss: 0.0583 - val_accuracy: 0.9870
Epoch 7/20
94/94 [==============================] - 1s 11ms/step - loss: 0.0190 - accuracy: 0.9952 - val_loss: 0.0346 - val_accuracy: 0.9900
Epoch 8/20
9

In [17]:
# Evaluate the model
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(X_test)
print(classification_report(Y_test, predictions.round()))
print(confusion_matrix(Y_test, predictions.round()))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       733
           1       0.98      0.97      0.98       267

    accuracy                           0.99      1000
   macro avg       0.99      0.98      0.98      1000
weighted avg       0.99      0.99      0.99      1000

[[729   4]
 [  9 258]]


In [18]:
# Save the model for later use
model.save("ShipCNN.h5")

In [20]:
# Load a model
new_model = tf.keras.models.load_model("ShipCNN.h5")
